In [ ]:
!unzip drive/MyDrive/data/cropped.zip -d cropped

In [ ]:
!pip install tqdm==4.64.1
!pip install dlib
!pip install face_recognition

In [4]:
import cv2
import os
#import face_recognition
import numpy as np
from tqdm import tqdm
import pandas as pd
import json

In [10]:
cap = cv2.VideoCapture('drive/MyDrive/bee.mp4')
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
  fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
else :
  fps = cap.get(cv2.CAP_PROP_FPS)

frame_count = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
print("Frames: " + str(frame_count))
print("Frames per second: " + str(fps))
print("lenght: " + str(frame_count/fps))
cap.release()


Frames: 131664
Frames per second: 29.97002997002997
lenght: 4393.1888


In [ ]:
# split video into frames

video = cv2.VideoCapture('bee.mp4')

fps = video.get(cv2.CAP_PROP_FPS)

num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

!mkdir frames

for i in tqdm(range(num_frames)):

    ret, frame = video.read()

    if not ret:
        break
    if not os.path.exists(f'frames/frame_{i}.jpg'):
      cv2.imwrite(f'frames/frame_{i}.jpg', frame)

video.release()


In [ ]:
# then find landmarks and using face_detection
# im going to split it into chunks 0-20000, 20000-40000, 40000-60000, 60000-80000, 80000-100000, 100000-120000, 120000-131664
# hard coded

!mkdir cropped

def crop(points):
  x_min = min(point[0] for point in points)
  x_max = max(point[0] for point in points)
  y_min = min(point[1] for point in points)
  y_max = max(point[1] for point in points)
  return x_min, y_min, x_max, y_max

frame_count = 131664

for i in tqdm(range(0, 20000)):
  num = str(i+1)
  for _ in range(len(str(frame_count))-len(num)):
    num = '0'+ num
  frame = cv2.imread(f"frames/frame_{i}.jpg")
  face_landmarks_list = face_recognition.face_landmarks(frame)
  if face_landmarks_list != []:
    [x, y, w, h] = crop(face_landmarks_list[0]['top_lip'] + face_landmarks_list[0]['bottom_lip'])
    frame = frame[y:h, x:w]
  if not os.path.exists(f"cropped/{num}.jpg"):
    cv2.imwrite(f"cropped/{num}.jpg", frame)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r cropped.zip cropped/
!cp cropped.zip "/content/drive/MyDrive/data/cropped.zip"

In [5]:
# then align each frame a with phoneme
align = json.load(open("drive/MyDrive/data/align.json"))['words']
aligned = []

for _d in range(len(align)):
  d = align[_d]
  if d["case"] == "success":
    if _d > 0 and align[_d-1]["case"] == "success" and d["start"] - align[_d-1]["end"] != 0:
      aligned.append({"duration":d["start"] - align[_d-1]["end"],"phone":"none","start":align[_d-1]["end"],"end":d["start"],"case":"success"})
    phones = d["phones"]
    for p in range(len(phones)):
      if p == 0:
        phones[p]["start"] = d["start"]
        phones[p]["end"] = phones[p]["start"] + phones[p]["duration"]
      else:
        phones[p]["start"] = phones[p-1]["end"]
        phones[p]["end"] = phones[p]["start"] + phones[p]["duration"]
      phones[p]["case"] = "success"
      phones[p]["phone"] = phones[p]["phone"].split("_")[0]
      phones[p]["duration"] = phones[p]["end"] - phones[p]["start"]
      if phones[p]["phone"] == "oov":
        phones[p]["case"] = "failed"
        phones[p]["phone"] = "none"
    for i in phones:
      aligned.append(i)
  elif align[_d]["case"] != "success" and align[_d-1]["case"] == "success" and _d != 0:
    index = _d
    while True:
      index += 1
      if align[index]["case"] == "success":
        aligned.append({"duration":align[index]["start"] - align[_d-1]["end"],"phone":"none","start":align[_d-1]["end"],"end":align[index]["start"],"case":"failed"})
        break
  elif align[_d]["case"] != "success" and _d == 0:
    index = _d
    while True:
      index += 1
      if align[index]["case"] == "success":
        aligned.append({"duration":align[index]["start"],"phone":"none","start":0,"end":align[index]["start"],"case":"failed"})
        break

pd.DataFrame(aligned)

,duration,phone,start,end,case
0,0.14,ah,1.75,1.89,success
1,0.15,k,1.89,2.04,success
2,0.10,ao,2.04,2.14,success
3,0.01,r,2.14,2.15,success
4,0.07,d,2.15,2.22,success
...,...,...,...,...,...
36025,0.06,ao,4391.19,4391.25,success
36026,0.08,r,4391.25,4391.33,success
36027,0.09,dh,4391.33,4391.42,success
36028,0.16,ae,4391.42,4391.58,success


from matplotlib import pyplot as plt
_df_0['duration'].plot(kind='hist', bins=20, title='duration')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['start'].plot(kind='hist', bins=20, title='start')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['end'].plot(kind='hist', bins=20, title='end')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('case').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_29['case'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_29, x='duration', y='case', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_30['case'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_30, x='start', y='case', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_31['case'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_31, x='end', y='case', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
frame_phone = []

for i in tqdm(range(frame_count)):
  now = {}
  num = str(i+1)
  for _ in range(len(str(frame_count))-len(num)):
    num = '0'+ num
  num = f'cropped/cropped/{num}' + '.jpg'
  time = (i+1)/fps
  for a in aligned:
    if a["start"] <= time and a["end"] > time:
      now["image"] = num
      now["phone"] = a["phone"]
      now["case"] = a["case"]
  frame_phone.append(now)

df = pd.DataFrame(frame_phone)
df.to_csv('drive/MyDrive/data/data.csv')
df

 66%|██████▌   | 86686/131664 [11:06<09:38, 77.80it/s]